In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import random

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving framingham_heart_disease.csv to framingham_heart_disease.csv


In [ ]:
data = pd.read_csv("framingham_heart_disease.csv")
X_train = np.array(data.drop("TenYearCHD" , axis = 1).copy())
Y_train = np.array([data['TenYearCHD'].copy()]).T
X_train = X_train.T
Y_train = Y_train.T
for i in range(0,X_train.shape[0]) :
  for j in range(0,X_train.shape[1]) :
    if np.isnan(X_train[i,j]) == True :
      X_train[i,j] = 0
while True :
  i = random.randint(0,Y_train.shape[1] - 1)
  if Y_train[0,i] == 0 :
    X_train = np.delete(X_train,i,1)
    Y_train = np.delete(Y_train,i,1)
  if Y_train.shape[1] < 1289 :
    break
def sigmoid(z) :
  return (1/(1+np.exp(-z)))
def relu(X) :
  X[X<0]=0
  return X
mean = np.array([np.mean(X_train,axis = 1)]).T
#max = np.array([np.max(X_train,axis = 1)]).T
var = np.array([np.mean(X_train*X_train,axis = 1)]).T
X_train = ((X_train - mean)/np.sqrt(var))
 
print(X_train.shape)
print(Y_train.shape)
n = X_train.shape[0]
m = X_train.shape[1]

(15, 1288)
(1, 1288)


In [ ]:
def initialize_wts(struct) :
  W = {}
  b = {}
  for i in range(1,len(struct)) :
    W["W" + str(i)] = np.random.randn(struct[i],struct[i-1])*np.sqrt(2/struct[i-1])
    #W["W" + str(i)] = np.random.randn(struct[i],struct[i-1])*0.01
    #W["W" + str(i)] = np.random.rand(struct[i],struct[i-1])
    b["b" + str(i)] = np.zeros((struct[i],1))
  return W,b
def forward_prop(X_train,W,b) :
  return relu(np.dot(W,X_train) + b)

In [ ]:
def model(struct,X_train,Y_train,lr,num_itr) :
  n = X_train.shape[0]
  m = X_train.shape[1]
  W,b = initialize_wts(struct)
  costu = []
  cost1 = 50
  cost2 = 100
  for k in range(0,num_itr) :
    if cost1 < 0.47 :
      if float(cost1) - float(cost2) > 0.00001 :
        lr = lr/7.5
      else :
        lr = lr*1.15
    cost2 = np.copy(cost1)
    A = {}
    A["A" + str(0)] = np.copy(X_train)
    for i in range(1,len(struct)-1) :
      A["A" + str(i)] = forward_prop(A["A" + str(i-1)],W["W" + str(i)],b["b" + str(i)])
    A["A" + str(len(struct) - 1)] = sigmoid(np.dot(W["W" + str(len(struct) - 1)] , A["A" + str(len(struct) - 2)]) + b["b" + str(len(struct) - 1)])
    dW = {}
    db = {}
    dZ = {}
    dZ["dZ" + str(len(struct) - 1)] = A["A" + str(len(struct) - 1)] - Y_train
    dW["dW" + str(len(struct) - 1)] = (1/m)*(np.dot(dZ["dZ" + str(len(struct) - 1)],A["A" + str(len(struct) - 2)].T))
    db["db" + str(len(struct) - 1)] = (1/m)*(np.sum(dZ["dZ" + str(len(struct) - 1)],axis = 1, keepdims = True))
    for i in range(0,len(W) - 1) :
      dZ["dZ" + str(len(W) - 1 - i)] = np.dot(W["W" + str(len(W) - i)].T,dZ["dZ" + str(len(W) - i)])*np.int64(A["A" + str(len(W) - 1 - i)] > 0)
      dW["dW" + str(len(W) - 1 - i)] = (1/m)*np.dot(dZ["dZ" + str(len(W) - 1 - i)] , A["A" + str(len(W) - 1 - i - 1)].T)
      db["db" + str(len(W) - i - 1)] = (1/m)*np.sum(dZ["dZ" + str(len(W) - 1 - i)],axis = 1,keepdims = True)
    for i in range(1,len(W) + 1) :
      W["W" + str(i)] = W["W" + str(i)] - lr*dW["dW" + str(i)]
      b["b" + str(i)] = b["b" + str(i)] - lr*db["db" + str(i)]
    cost1 = (-1/m)*(np.dot(Y_train,np.log(A["A" + str(len(struct) - 1)]).T) + np.dot(1 - Y_train,np.log(1 - A["A" + str(len(struct) - 1)]).T))
    costu.append(float(cost1))
    print("Cost after {}th iteration is : {} {}".format(k,cost1,lr))
  plt.plot(list(range(num_itr)) , costu)
  plt.show()
  return W,b,A["A" + str(len(struct) - 1)]

In [ ]:
W,b,A = model([n,13,11,7,5,3,1] , X_train , Y_train , 0.2 ,15000)

In [ ]:
pred = np.zeros(A.shape)
for i in range(0,A.shape[1]) :
  if A[0,i] > 0.50 :
    pred[0,i] = 1
fai = np.absolute(pred - Y_train)
print(np.sum(fai))
accuracy = 1 - (np.sum(fai)/Y_train.shape[1])
print(accuracy*100)

379.0
70.57453416149069


In [ ]:
i = 5
for i in reversed(range(0,8)) :
  print(i)